In [46]:
from ultralytics import YOLO
from utils import (
    select_device,
    process_yolo_result,
    process_yolo_result_ocr,
    rectify,
    YoloJSONRequest,
    YoloJSONRequest2,
    ModelJSONRequest,
    UnetJSONRequest,
    PlatePipelineRequest,
)
from my_models import (
    Plate_Unet,
)
from torchvision import transforms
from model_handler import (
    YoloType,
    UNetType
)
from PIL import Image
import numpy as np
import torch
import base64
import io

In [47]:
image = Image.open("/home/al-fazli/Desktop/truck2.jpeg")

In [48]:
my_models = {}
my_transforms = {}

In [49]:
device = select_device()
my_models["yolo_plate"] = YOLO(YoloType.CustomPlate.Plate_best.value)
my_models["plate_unet"] = Plate_Unet(UNetType.Corner_best, device=device)
my_models["yolo_ocr"] = YOLO(YoloType.CustomPlateOCR.plate_ocr_best.value)
my_transforms["unet"] = transforms.Compose(
    [
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean = [0.485, 0.456, 0.406],
            std = [0.229, 0.224, 0.225],
        )
    ]
)

NVIDIA GeForce RTX 4090 have been located and selected


In [116]:
conf_threshold_bb = 0.7
conf_threshold_ocr = 0.7
return_plates_locations = True
return_base64_cropped_plates = True
return_base64_rectified_plates = True
return_base64_ocr_plate_results = True
response = {}

In [120]:
%%timeit

plates_res = my_models["yolo_plate"](
    source = image,
    conf = conf_threshold_bb,
    verbose = False,
)

plate_response = process_yolo_result(plates_res[0])

9.87 ms ± 26 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [118]:
plates_res = my_models["yolo_plate"](
    source = image,
    conf = conf_threshold_bb,
    verbose = False,
)

plate_response = process_yolo_result(plates_res[0])

In [121]:
%%timeit

if return_plates_locations:
    response = {
        "bounding_box_data": plate_response,
        "origin_image_size": {
            "x": plates_res[0].orig_shape[1],
            "y": plates_res[0].orig_shape[0],
        }
    }

73.1 ns ± 0.255 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [122]:
if return_plates_locations:
    response = {
        "bounding_box_data": plate_response,
        "origin_image_size": {
            "x": plates_res[0].orig_shape[1],
            "y": plates_res[0].orig_shape[0],
        }
    }

In [123]:
%%timeit

boxes = plate_response[0]["boxes"]
plates = [None] * len(boxes)
for i, box in enumerate(boxes):
    _, x1, y1, x2, y2 = map(int, box.values())
    plates[i] = np.array(image)[y1:y2, x1:x2, :]
    plates[i] = Image.fromarray(plates[i])

1.41 ms ± 4.47 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [124]:
boxes = plate_response[0]["boxes"]
plates = [None] * len(boxes)
for i, box in enumerate(boxes):
    _, x1, y1, x2, y2 = map(int, box.values())
    plates[i] = np.array(image)[y1:y2, x1:x2, :]
    plates[i] = Image.fromarray(plates[i])

In [125]:
%%timeit

if return_base64_cropped_plates:
    plates_base64 = [None] * len(plates)
    for i, plate in enumerate(plates):
        plates_base64[i] = plate
        buffer = io.BytesIO()
        plates_base64[i].save(buffer, format="PNG")
        buffer.seek(0)
        plates_base64[i] = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response["base64_cropped_plates"] = plates_base64

1.88 ms ± 4.13 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [126]:
if return_base64_cropped_plates:
    plates_base64 = [None] * len(plates)
    for i, plate in enumerate(plates):
        plates_base64[i] = plate
        buffer = io.BytesIO()
        plates_base64[i].save(buffer, format="PNG")
        buffer.seek(0)
        plates_base64[i] = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response["base64_cropped_plates"] = plates_base64

In [127]:
%%timeit

torch_plates = [my_transforms["unet"](plate) for plate in plates]
torch_plates = torch.stack(torch_plates).to(device)

985 μs ± 1.9 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [128]:
torch_plates = [my_transforms["unet"](plate) for plate in plates]
torch_plates = torch.stack(torch_plates).to(device)

In [130]:
%%timeit

plates_segmentation_results = my_models["plate_unet"](torch_plates)

1.65 ms ± 4.05 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [131]:
plates_segmentation_results = my_models["plate_unet"](torch_plates)

In [132]:
%%timeit 

segment_arrays = [
    segment.squeeze(0).cpu().numpy()
    for segment in plates_segmentation_results 
]
rectified_plates = [
    rectify(
        image = plates[i],
        segmentation_result = segment_array 
    )
    for i, segment_array in enumerate(segment_arrays)
]
rectified_plates = [
    Image.fromarray(rectified_plate)
    for rectified_plate in rectified_plates
]

317 μs ± 798 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [133]:
segment_arrays = [
    segment.squeeze(0).cpu().numpy()
    for segment in plates_segmentation_results 
]
rectified_plates = [
    rectify(
        image = plates[i],
        segmentation_result = segment_array 
    )
    for i, segment_array in enumerate(segment_arrays)
]
rectified_plates = [
    Image.fromarray(rectified_plate)
    for rectified_plate in rectified_plates
]

In [134]:
%%timeit

if return_base64_rectified_plates:
    rectified_plates_base64 = [None] * len(rectified_plates)
    for i, rectified_plate in enumerate(rectified_plates):
        rectified_plates_base64[i] = rectified_plate
        buffer = io.BytesIO()
        rectified_plates_base64[i].save(buffer, format="PNG")
        buffer.seek(0)
        rectified_plates_base64[i] = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response["base64_rectified_plates"] = rectified_plates_base64

2.19 ms ± 2.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [135]:
if return_base64_rectified_plates:
    rectified_plates_base64 = [None] * len(rectified_plates)
    for i, rectified_plate in enumerate(rectified_plates):
        rectified_plates_base64[i] = rectified_plate
        buffer = io.BytesIO()
        rectified_plates_base64[i].save(buffer, format="PNG")
        buffer.seek(0)
        rectified_plates_base64[i] = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response["base64_rectified_plates"] = rectified_plates_base64

In [137]:
%%timeit

ocr_res = my_models["yolo_ocr"](
    source = rectified_plates,
    conf = conf_threshold_ocr,
    verbose = False,
)
ocr_results = [
    process_yolo_result_ocr(ocr)
    for ocr in ocr_res
]

response["ocr_result"] = ocr_results

17.9 ms ± 37.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [138]:
ocr_res = my_models["yolo_ocr"](
    source = rectified_plates,
    conf = conf_threshold_ocr,
    verbose = False,
)
ocr_results = [
    process_yolo_result_ocr(ocr)
    for ocr in ocr_res
]

response["ocr_result"] = ocr_results

In [139]:
%%timeit

if return_base64_ocr_plate_results:
    base64_ocr_plate_results = [None] * len(ocr_res)
    for i, ocr in enumerate(ocr_res):
        base64_ocr_plate_results[i] = Image.fromarray(ocr.plot()[:, :, ::-1])
        buffer = io.BytesIO()
        base64_ocr_plate_results[i].save(buffer, format="PNG")
        buffer.seek(0)
        base64_ocr_plate_results[i] = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response["base64_ocr_results"] = base64_ocr_plate_results

5.56 ms ± 35.5 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [92]:
if return_base64_ocr_plate_results:
    base64_ocr_plate_results = [None] * len(ocr_res)
    for i, ocr in enumerate(ocr_res):
        base64_ocr_plate_results[i] = Image.fromarray(ocr.plot()[:, :, ::-1])
        buffer = io.BytesIO()
        base64_ocr_plate_results[i].save(buffer, format="PNG")
        buffer.seek(0)
        base64_ocr_plate_results[i] = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response["base64_ocr_results"] = base64_ocr_plate_results

In [143]:
%%timeit

conf_threshold_bb = 0.7
conf_threshold_ocr = 0.7
return_plates_locations = False
return_base64_cropped_plates = False
return_base64_rectified_plates = False
return_base64_ocr_plate_results = False

response = {}

plates_res = my_models["yolo_plate"](
    source = image,
    conf = conf_threshold_bb,
    verbose = False,
)

plate_response = process_yolo_result(plates_res[0])

if return_plates_locations:
    response = {
        "bounding_box_data": plate_response,
        "origin_image_size": {
            "x": plates_res[0].orig_shape[1],
            "y": plates_res[0].orig_shape[0],
        }
    }

boxes = plate_response[0]["boxes"]
plates = [None] * len(boxes)
for i, box in enumerate(boxes):
    _, x1, y1, x2, y2 = map(int, box.values())
    plates[i] = np.array(image)[y1:y2, x1:x2, :]
    plates[i] = Image.fromarray(plates[i])

if return_base64_cropped_plates:
    plates_base64 = [None] * len(plates)
    for i, plate in enumerate(plates):
        plates_base64[i] = plate
        buffer = io.BytesIO()
        plates_base64[i].save(buffer, format="PNG")
        buffer.seek(0)
        plates_base64[i] = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response["base64_cropped_plates"] = plates_base64

torch_plates = [my_transforms["unet"](plate) for plate in plates]
torch_plates = torch.stack(torch_plates).to(device)

plates_segmentation_results = my_models["plate_unet"](torch_plates)

segment_arrays = [
    segment.squeeze(0).cpu().numpy()
    for segment in plates_segmentation_results 
]
rectified_plates = [
    rectify(
        image = plates[i],
        segmentation_result = segment_array 
    )
    for i, segment_array in enumerate(segment_arrays)
]
rectified_plates = [
    Image.fromarray(rectified_plate)
    for rectified_plate in rectified_plates
]

if return_base64_rectified_plates:
    rectified_plates_base64 = [None] * len(rectified_plates)
    for i, rectified_plate in enumerate(rectified_plates):
        rectified_plates_base64[i] = rectified_plate
        buffer = io.BytesIO()
        rectified_plates_base64[i].save(buffer, format="PNG")
        buffer.seek(0)
        rectified_plates_base64[i] = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response["base64_rectified_plates"] = rectified_plates_base64

ocr_res = my_models["yolo_ocr"](
    source = rectified_plates,
    conf = conf_threshold_ocr,
    verbose = False,
)
ocr_results = [
    process_yolo_result_ocr(ocr)
    for ocr in ocr_res
]

response["ocr_result"] = ocr_results

if return_base64_ocr_plate_results:
    base64_ocr_plate_results = [None] * len(ocr_res)
    for i, ocr in enumerate(ocr_res):
        base64_ocr_plate_results[i] = Image.fromarray(ocr.plot()[:, :, ::-1])
        buffer = io.BytesIO()
        base64_ocr_plate_results[i].save(buffer, format="PNG")
        buffer.seek(0)
        base64_ocr_plate_results[i] = base64.b64encode(buffer.getvalue()).decode('utf-8')
    response["base64_ocr_results"] = base64_ocr_plate_results

33.4 ms ± 109 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
